In [75]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from itertools import combinations
from sklearn.metrics import roc_curve, roc_auc_score
import json

## B1: Đọc dữ liệu và bổ sung một số feature

### Dữ liệu dùng cho buổi training: https://www.kaggle.com/c/home-credit-default-risk/overview
### Chú ý: Laptop phải được cài Python 3 và scikit learn library

In [76]:
dfDev =  pd.read_csv('application_train.csv')
dfDev['DSR']= round(dfDev['AMT_ANNUITY']/dfDev['AMT_INCOME_TOTAL'],2)
dfDev['LTV']=round(dfDev['AMT_CREDIT']/dfDev['AMT_GOODS_PRICE'],2)

dfDev['YRS_EMPLOYED'] = round(dfDev['DAYS_EMPLOYED']*-1/365,2)
dfDev['AGE'] = round(dfDev['DAYS_BIRTH']*-1/365,2)
dfDev['YRS_LAST_PHONE_CHANGE'] = round(dfDev['DAYS_LAST_PHONE_CHANGE']*-1/365,2)
dfDev['YRS_REGISTRATION'] = round(dfDev['DAYS_REGISTRATION']*-1/365,2)
dfDev['YRS_ID_PUBLISH'] = round(dfDev['DAYS_ID_PUBLISH']*-1/365,2)

In [77]:
dfDev.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DSR,LTV,YRS_EMPLOYED,AGE,YRS_LAST_PHONE_CHANGE,YRS_REGISTRATION,YRS_ID_PUBLISH
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,1.0,0.12,1.16,1.75,25.92,3.11,9.99,5.81
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.13,1.15,3.25,45.93,2.27,3.25,0.80
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.10,1.00,0.62,52.18,2.23,11.67,6.93
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,0.22,1.05,8.33,52.07,1.69,26.94,6.68
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.18,1.00,8.32,54.61,3.03,11.81,9.47


In [78]:
dfDev.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'DSR', 'LTV', 'YRS_EMPLOYED', 'AGE',
       'YRS_LAST_PHONE_CHANGE', 'YRS_REGISTRATION', 'YRS_ID_PUBLISH'],
      dtype='object', length=129)

In [79]:
dfDev = dfDev.sample(frac=0.1, random_state=2)

In [80]:
dfDev.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DSR,LTV,YRS_EMPLOYED,AGE,YRS_LAST_PHONE_CHANGE,YRS_REGISTRATION,YRS_ID_PUBLISH
49103,156852,0,Cash loans,F,Y,N,3,126000.0,888840.0,32053.5,...,0.0,0.0,1.0,0.25,1.32,1.98,30.75,4.09,14.24,6.92
96031,211489,0,Cash loans,M,Y,N,0,225000.0,360000.0,30798.0,...,0.0,0.0,7.0,0.14,1.00,6.45,49.33,4.35,13.19,3.99
300915,448618,0,Cash loans,F,N,Y,0,126000.0,312768.0,23382.0,...,0.0,0.0,1.0,0.19,1.16,-1000.67,61.87,1.61,13.97,13.97
260145,401051,0,Cash loans,F,N,N,0,225000.0,1096020.0,55962.0,...,0.0,0.0,0.0,0.25,1.22,7.05,50.88,5.65,22.43,5.73
99054,215007,0,Cash loans,M,N,Y,0,270000.0,1288350.0,37800.0,...,0.0,0.0,9.0,0.14,1.15,0.34,36.27,1.87,20.17,11.08


## Function để tính Badrate, AR
Chúng ta sẽ chia các dữ liệu vào các bucket cố định thông thông qua tham số bucket. Nếu là số thì sẽ chia đều thành từng ấy bin, nếu là list thì chia theo các khoảng cắt đó, nếu là dict thì dựa vào key để chia. Dict thì thường áp dụng cho muốn chia bin theo categoty

In [81]:
def bucketingDataset(dataset, keyfields, fieldname, bucket):
    keyfields.append(fieldname)
    df=dataset[keyfields].copy()

    if df[fieldname].dtype == 'object':
        df[fieldname] = df[fieldname].fillna('Missing')
    else:
        df[fieldname] = df[fieldname].fillna(-999999)
        
    if type(bucket) == int:
        if (df[fieldname].dtype == 'object') or (len(df[fieldname].value_counts())<=10):
            df['BucketTmp'] = df[fieldname]
        else:
            df['BucketTmp'] = pd.qcut(df[fieldname],q=bucket, duplicates='drop') #.astype(str)
    elif type(bucket) == list: 
        df['BucketTmp'] = pd.cut(df[fieldname], bucket)
    elif type(bucket) == dict:
        mymap = {v:k for k,vv in bucket.items() for v in vv}
        df['BucketTmp'] = df[fieldname].map(mymap)
    else:
        df['BucketTmp'] = df[fieldname]
    
    dfGrp=df.groupby('BucketTmp')['SK_ID_CURR'].agg(Total='count')
    todrop = dfGrp[dfGrp['Total']<=0]
    dfGrp.drop(todrop.index, axis=0, inplace=True)

    dfGrp.reset_index(inplace=True)
    dfGrp['Bucket'] = dfGrp.index.astype(str).str.zfill(2) + '. ' + dfGrp['BucketTmp'].astype(str)
    
    df = df.merge(dfGrp, on='BucketTmp')
    df.drop('BucketTmp', axis=1, inplace=True)
    
    return df

def RiskRankingTable(dataset, fieldname, bucket='none'):
# Pass a dataset, with 
#   fieldname: measures, i.e. score, factor value,
#   target: good/bad
#   bucket: - an integer: use qcut to cut by percentile
#             - a list [0, .25, 5, ...]: use cut to cut
#             - a dict {a=['a1', 'a2'],b=['b1','b2']}: map value to each bucket

    df= bucketingDataset(dataset, ['SK_ID_CURR','TARGET'], fieldname, bucket)

    
    df=df.groupby('Bucket')['TARGET'].agg(Total='count',Bad='sum')
    df= df.reset_index()

    df.insert(0,'FactorName',fieldname)

    df['TotalAll'] = df['Total'].sum()
    df['BadAll'] = df['Bad'].sum()
    df['BadRateAll'] = df['BadAll']/df['TotalAll']
    df['Dist'] = df['Total']/df['TotalAll']
    df['BadRate'] = df['Bad']/df['Total']
    df['RBR'] = df['BadRate']/df['BadRateAll']
    
    df.drop('TotalAll', axis=1, inplace=True)
    df.drop('BadAll', axis=1, inplace=True)
    df.drop('BadRateAll', axis=1, inplace=True)
    
    return df

def AR_Calculator(dataset, totalfield, badfield, sortbyfield='none'):
    if sortbyfield=='none':
        dfAR = dataset[[totalfield, badfield]].copy()
    else:
        dfAR = dataset[[totalfield, badfield,sortbyfield]].copy()
        dfAR.sort_values(sortbyfield, ascending=False, inplace=True)
    
    nTotal = dfAR[totalfield].sum()
    nBad = dfAR[badfield].sum()

    iCummDist=0
    iCummBR=0
    prevCummDist = 0
    prevCummBR = 0
    iAUC = 0

    dfAR.loc[:,'CummTotal'] = 0
    dfAR.loc[:,'CummBad'] = 0

    for i, row in dfAR.iterrows():
        iCummDist += row[totalfield]/nTotal
        iCummBR += row[badfield]/nBad

        iAUC += (iCummDist - prevCummDist)*( iCummBR + prevCummBR)/2

        prevCummDist=iCummDist
        prevCummBR=iCummBR

        dfAR.loc[i, 'CummTotal'] = iCummDist
        dfAR.loc[i, 'CummBad'] = iCummBR

    AR = (iAUC-0.5)/((1-nBad/nTotal/2)-0.5)
    
    return AR, dfAR

## Generate Longlist Factor AR and Distribution

In [82]:
dflonglist = pd.read_csv('LongListFactor_sample.csv')
dfResult = dflonglist[['FactorCategory', 'FactorName']].copy()
                      
dfResult['Total']  = 0
dfResult['Bad'] = 0
dfResult['BadRate'] = 0
dfResult['AR'] = 0
dfResult['AR_MAX'] = 0

for i, row in dflonglist.iterrows():
    dfFactor = RiskRankingTable(dfDev,row['FactorName'] , bucket=20)
    dfFactor.insert(0,'FactorCategory', row['FactorCategory'])
    if i == 0:
        dfFactorDetail = dfFactor.copy()
    else:
        dfFactorDetail = dfFactorDetail.append(dfFactor,ignore_index = True)
        
    AR, dfAR = AR_Calculator(dfFactor, 'Total', 'Bad', 'none')
    AR_MAX, dfAR = AR_Calculator(dfFactor, 'Total', 'Bad', 'BadRate')
  
    nTotal = dfAR['Total'].sum()
    nBad = dfAR['Bad'].sum()
    
    dfResult.loc[i,'Total']=nTotal
    dfResult.loc[i,'Bad']=nBad
    dfResult.loc[i,'BadRate']=nBad/nTotal
    dfResult.loc[i,'AR']=AR
    dfResult.loc[i,'AR_MAX']=AR_MAX
    
dfFactorDetail.to_csv(f"LongListFactor_Detail.csv", index=False)
dfResult.to_csv('LongListFactor_Result.csv', index=False)

In [83]:
dfResult

,FactorCategory,FactorName,Total,Bad,BadRate,AR,AR_MAX
0,Application,NAME_CONTRACT_TYPE,30751,2426,0.078892,0.029355,0.029355
1,Demograhic,CODE_GENDER,30751,2426,0.078892,-0.091126,0.091126
2,Demograhic,FLAG_OWN_CAR,30751,2426,0.078892,0.048789,0.048789
3,Demograhic,FLAG_OWN_REALTY,30751,2426,0.078892,-0.003524,0.003524
4,Demograhic,CNT_CHILDREN,30751,2426,0.078892,-0.039691,0.040470
...,...,...,...,...,...,...,...
117,CB,AMT_REQ_CREDIT_BUREAU_DAY,30751,2426,0.078892,0.045062,0.049663
118,CB,AMT_REQ_CREDIT_BUREAU_WEEK,30751,2426,0.078892,0.051121,0.051128
119,CB,AMT_REQ_CREDIT_BUREAU_MON,30751,2426,0.078892,-0.001211,0.015760
120,CB,AMT_REQ_CREDIT_BUREAU_QRT,30751,2426,0.078892,0.055474,0.067055


## Gom nhóm lại các bucket

Following steps:
1. Trong file analysis/ScorecardDevelopement_copy.xlsm mình sẽ định nghĩa gộp các bin đã được sinh ra ở trên 
2. Danh sách các biến được chọn vào shortlist được liệt kê ở sheet 'Longlist_result'
2. Chạy hàm RiskRankingTable để tính lại badrate của các bin trong shortlist

In [84]:
import json

dfShortlist = pd.read_excel('ScorecardDevelopement_wip.xlsm',sheet_name='Longlist_Result')
dfShortlist = dfShortlist[dfShortlist['isSelected']=='Y'][['FactorCategory','FactorName','ShortlistCut']].reset_index(drop=True)
for i, row in dfShortlist.iterrows():
    sBucket = 'none' if (row['ShortlistCut']=='none') else json.loads(row['ShortlistCut'])
    df = RiskRankingTable(dfDev, row['FactorName'], bucket=sBucket)
    if (i==0):
        dfShortlistedDetail = df.copy()
    else:
        dfShortlistedDetail = dfShortlistedDetail.append(df,ignore_index = True)

dfShortlistedDetail.to_csv(f"ShortListFactor_Detail.csv", index=False)

In [85]:
dfShortlistedDetail

,FactorName,Bucket,Total,Bad,Dist,BadRate,RBR
0,NAME_CONTRACT_TYPE,00. Cash loans,27752,2255,0.902475,0.081255,1.029961
1,NAME_CONTRACT_TYPE,01. Revolving loans,2999,171,0.097525,0.057019,0.722750
2,CODE_GENDER,00. F,20365,1403,0.662255,0.068893,0.873256
3,CODE_GENDER,01. M,10386,1023,0.337745,0.098498,1.248521
4,LTV,"00. (-9999999.0, 0.0]",22,1,0.000715,0.045455,0.576164
...,...,...,...,...,...,...,...
166,TOTALAREA_MODE,"07. (0.07, 0.086]",1618,113,0.052616,0.069839,0.885255
167,TOTALAREA_MODE,"08. (0.086, 0.11]",1491,96,0.048486,0.064386,0.816135
168,TOTALAREA_MODE,"09. (0.11, 0.162]",1646,96,0.053527,0.058323,0.739282
169,TOTALAREA_MODE,"10. (0.162, 0.23]",1480,78,0.048129,0.052703,0.668038


## Normalised điểm từng bucket

In [86]:
def computeNormalisedScore(dfBucketMap):
    dfNorm = dfBucketMap.groupby(['FactorName','FinalBucketName'])['Total','Bad', 'Dist'].agg('sum').reset_index()
    dfNorm['Odd'] = dfNorm['Bad']/(dfNorm['Total']-dfNorm['Bad'])
    dfNorm['LogOdd'] = np.log(dfNorm['Odd'])
    dfNorm['LogOdd_wt'] = dfNorm['LogOdd']*dfNorm['Dist']

    dfLogOdd = dfNorm.groupby(['FactorName'])['LogOdd_wt'].agg(LogOdd_sum = 'sum').reset_index()
    dfNorm = dfNorm.merge(dfLogOdd, on='FactorName', how='left')
    dfNorm['SD'] = (dfNorm['LogOdd'] - dfNorm['LogOdd_sum'])**2*dfNorm['Dist']
    dfSD = dfNorm.groupby(['FactorName'])['SD'].agg(SD_sum = 'sum').reset_index()
    dfSD['SD_sqrt'] = dfSD['SD_sum']**(0.5)

    dfNorm = dfNorm.merge(dfSD, on='FactorName', how='left')
    dfNorm['NormScore'] = round((-50)*((dfNorm['LogOdd'] - dfNorm['LogOdd_sum'])/dfNorm['SD_sqrt']),2)
    dfNorm = dfNorm[['FactorName','FinalBucketName', 'NormScore']]
    return dfNorm

#Compute Normalised Score
dfShortlistedDetail = pd.read_excel('ScorecardDevelopement_wip.xlsm',sheet_name='Shortlist_Detail')
mapNormScore = computeNormalisedScore(dfShortlistedDetail)


<ipython-input-86-22437b4b22c1>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfNorm = dfBucketMap.groupby(['FactorName','FinalBucketName'])['Total','Bad', 'Dist'].agg('sum').reset_index()


In [87]:
mapNormScore

,FactorName,FinalBucketName,NormScore
0,AGE,01. <=30,-62.20
1,AGE,"02. (30,50]",-25.16
2,AGE,"03. (50,60)",51.63
3,AGE,04. 60<,94.41
4,APARTMENTS_AVG,01. missing,-39.73
5,APARTMENTS_AVG,"02. (0,0.13]",13.98
6,APARTMENTS_AVG,03. >0.13,106.21
7,CODE_GENDER,01. Female,35.71
8,CODE_GENDER,02. Male,-70.01
9,EXT_SOURCE_1,"01. (0, 0.4]",-65.84


# Tạo dữ liệu với các biến shortlist
bao gồm giá trị, bucket và norm score

In [88]:
dfShortlist = pd.read_excel('ScorecardDevelopement_wip.xlsm',sheet_name='Longlist_Result')
dfShortlist = dfShortlist[dfShortlist['isSelected']=='Y'][['FactorCategory','FactorName','ShortlistCut']].reset_index(drop=True)

dfShortlistedDetail = pd.read_excel('ScorecardDevelopement_wip.xlsm',sheet_name='Shortlist_Detail')
mapNormScore = computeNormalisedScore(dfShortlistedDetail)


for i, row in dfShortlist.iterrows():
    sBucket = 'none' if (row['ShortlistCut']=='none') else json.loads(row['ShortlistCut'])
    df = bucketingDataset(dfDev, ['SK_ID_CURR', 'TARGET'], row['FactorName'], bucket=sBucket)
    
    #merge to shortlistedDetail to get the final buckets in the dev template
    df = df.merge(dfShortlistedDetail[dfShortlistedDetail['FactorName']== row['FactorName']],on=['Bucket'])
    
    #merge to mapNormScore to get the normalised score
    df = df.merge(mapNormScore[(mapNormScore['FactorName']== row['FactorName'])],on=['FinalBucketName'])

    bucketname = row['FactorName'] + '_bucket'
    normscorename = row['FactorName'] + '_NS'
        
    df.rename(columns={'FinalBucketName':bucketname, 'NormScore': normscorename}, inplace=True)
    
    if (i==0):
        df =df[['SK_ID_CURR', 'TARGET',row['FactorName'], bucketname, normscorename]]
        dfNormDev = df.copy()
    else:
        df =df[['SK_ID_CURR',row['FactorName'], bucketname, normscorename]]
        dfNormDev = pd.merge(dfNormDev, df, on='SK_ID_CURR', how='left')

<ipython-input-86-22437b4b22c1>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfNorm = dfBucketMap.groupby(['FactorName','FinalBucketName'])['Total','Bad', 'Dist'].agg('sum').reset_index()


In [89]:
dfNormDev.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE_bucket,NAME_CONTRACT_TYPE_NS,CODE_GENDER,CODE_GENDER_bucket,CODE_GENDER_NS,LTV,LTV_bucket,...,APARTMENTS_AVG_NS,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_AVG_bucket,YEARS_BEGINEXPLUATATION_AVG_NS,FLOORSMAX_AVG,FLOORSMAX_AVG_bucket,FLOORSMAX_AVG_NS,TOTALAREA_MODE,TOTALAREA_MODE_bucket,TOTALAREA_MODE_NS
0,156852,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.32,03. >1.28 or missing,...,-39.73,-999999.0000,01. missing,-44.47,-999999.0000,01. missing,-38.74,-999999.0000,01. missing,-40.68
1,211489,0,Cash loans,01. Cash,-16.44,M,02. Male,-70.01,1.00,01. <=1.15,...,-39.73,-999999.0000,01. missing,-44.47,-999999.0000,01. missing,-38.74,-999999.0000,01. missing,-40.68
2,448618,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.16,"02. (1.15, 1.28]",...,13.98,0.9776,"02. (0, 0.98]",8.40,0.1667,"02. (0, 0.167]",4.18,0.0701,"03. (0.059, 0.11]",44.06
3,401051,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.22,"02. (1.15, 1.28]",...,13.98,0.9886,04. >0.986,91.83,0.3750,03. >0.167,94.62,0.1013,"03. (0.059, 0.11]",44.06
4,215007,0,Cash loans,01. Cash,-16.44,M,02. Male,-70.01,1.15,01. <=1.15,...,13.98,0.9732,"02. (0, 0.98]",8.40,0.1250,"02. (0, 0.167]",4.18,0.0319,"02. (0, 0.059]",-6.85


## Check Correlation

In [90]:
dfCorr = dfNormDev.filter(regex='_NS$',axis=1)
corr = dfCorr.corr()
corr.style.background_gradient(cmap='coolwarm')
# 'RdBu_r' & 'BrBG' are other good diverging colormaps

,NAME_CONTRACT_TYPE_NS,CODE_GENDER_NS,LTV_NS,NAME_INCOME_TYPE_NS,NAME_EDUCATION_TYPE_NS,AGE_NS,YRS_EMPLOYED_NS,YRS_ID_PUBLISH_NS,YRS_LAST_PHONE_CHANGE_NS,OCCUPATION_TYPE_NS,EXT_SOURCE_1_NS,EXT_SOURCE_2_NS,EXT_SOURCE_3_NS,APARTMENTS_AVG_NS,YEARS_BEGINEXPLUATATION_AVG_NS,FLOORSMAX_AVG_NS,TOTALAREA_MODE_NS
NAME_CONTRACT_TYPE_NS,1.000000,0.014114,0.230087,-0.015313,0.054892,-0.067994,-0.056833,-0.050847,-0.062980,0.012815,-0.004255,0.018646,-0.008079,0.017073,0.016411,0.019859,0.019066
CODE_GENDER_NS,0.014114,1.000000,0.002658,0.108001,0.014931,0.147209,0.164175,-0.004071,0.017008,0.234322,0.190002,0.010346,0.022524,0.024332,0.023183,0.015810,0.018927
LTV_NS,0.230087,0.002658,1.000000,0.046042,0.088190,0.028432,0.046185,0.014560,0.022000,0.073739,0.044159,0.070184,0.051798,0.032453,0.040079,0.045796,0.043536
NAME_INCOME_TYPE_NS,-0.015313,0.108001,0.046042,1.000000,0.074174,0.320599,0.338038,0.104038,0.017872,0.288590,0.072128,0.066504,0.048936,0.052954,0.035185,0.062097,0.056108
NAME_EDUCATION_TYPE_NS,0.054892,0.014931,0.088190,0.074174,1.000000,-0.110027,-0.068916,-0.017114,-0.000699,0.248319,0.119795,0.113684,-0.024445,0.111887,0.115825,0.137698,0.128242
AGE_NS,-0.067994,0.147209,0.028432,0.320599,-0.110027,1.000000,0.578586,0.309602,0.061416,0.223652,0.151773,0.059569,0.143872,0.020533,0.015730,0.024098,0.024751
YRS_EMPLOYED_NS,-0.056833,0.164175,0.046185,0.338038,-0.068916,0.578586,1.000000,0.208309,0.099896,0.306259,0.115678,0.045899,0.134320,0.008489,0.010830,0.007711,0.012010
YRS_ID_PUBLISH_NS,-0.050847,-0.004071,0.014560,0.104038,-0.017114,0.309602,0.208309,1.000000,0.084368,0.084527,0.035906,0.041078,0.112364,0.007034,0.005893,0.009667,0.010253
YRS_LAST_PHONE_CHANGE_NS,-0.062980,0.017008,0.022000,0.017872,-0.000699,0.061416,0.099896,0.084368,1.000000,0.019032,0.093502,0.149585,0.075342,0.035475,0.024566,0.034095,0.036675
OCCUPATION_TYPE_NS,0.012815,0.234322,0.073739,0.288590,0.248319,0.223652,0.306259,0.084527,0.019032,1.000000,0.156934,0.068992,0.033213,0.056556,0.056958,0.066419,0.059633


## Tạo kịch bản để chạy MFA
Kịch bản sẽ kết hợp các biến từ nhiều group có cùng tính chất với nhau. 

In [91]:
# Put factors into groups and set min and max number of factors to select
# After that run this steps, it will create dfCombi which keep all the combination 

grp1 = ['LTV_NS']

grp2 = ['CODE_GENDER_NS','AGE_NS', 'YRS_EMPLOYED_NS', 'OCCUPATION_TYPE_NS', 
        'NAME_EDUCATION_TYPE_NS', 'YRS_ID_PUBLISH_NS', 'NAME_INCOME_TYPE_NS', 'YRS_LAST_PHONE_CHANGE_NS']

grp3 = ['EXT_SOURCE_1_NS', 'EXT_SOURCE_2_NS', 'EXT_SOURCE_3_NS']

grp4 = ['APARTMENTS_AVG_NS','YEARS_BEGINEXPLUATATION_AVG_NS', 'FLOORSMAX_AVG_NS', 'TOTALAREA_MODE_NS']

def GroupCombi(grp, min_nbr, max_nbr):
    combi1 = pd.DataFrame({'CombiFactors': []})
    for i in range(min_nbr,max_nbr+1):
        combi = combinations(grp,i)
        for j in combi:
            combi1 = combi1.append({'CombiFactors':j}, ignore_index=True)
    return combi1

combi1 = GroupCombi(grp1, 1, 1)
combi2 = GroupCombi(grp2, 5, 6)
combi3 = GroupCombi(grp3, 2, 6)
combi4 = GroupCombi(grp4, 0, 1)

dfCombi=pd.DataFrame({'CombiFactors':[]})
for i in combi1['CombiFactors']:
    for j in combi2['CombiFactors']:
        for k in combi3['CombiFactors']:
            for l in combi4['CombiFactors']:
                dfCombi=dfCombi.append({'CombiFactors':i+j+k+l}, ignore_index=True)

In [92]:
dfCombi

,CombiFactors
0,"(LTV_NS, CODE_GENDER_NS, AGE_NS, YRS_EMPLOYED_..."
1,"(LTV_NS, CODE_GENDER_NS, AGE_NS, YRS_EMPLOYED_..."
2,"(LTV_NS, CODE_GENDER_NS, AGE_NS, YRS_EMPLOYED_..."
3,"(LTV_NS, CODE_GENDER_NS, AGE_NS, YRS_EMPLOYED_..."
4,"(LTV_NS, CODE_GENDER_NS, AGE_NS, YRS_EMPLOYED_..."
...,...
1675,"(LTV_NS, YRS_EMPLOYED_NS, OCCUPATION_TYPE_NS, ..."
1676,"(LTV_NS, YRS_EMPLOYED_NS, OCCUPATION_TYPE_NS, ..."
1677,"(LTV_NS, YRS_EMPLOYED_NS, OCCUPATION_TYPE_NS, ..."
1678,"(LTV_NS, YRS_EMPLOYED_NS, OCCUPATION_TYPE_NS, ..."


## Run MFA

In [93]:
def run_logistic_regression(dfNormScore, list_factors):
    x_sample = dfNormScore[list(list_factors)]
    y_sample = dfNormScore['TARGET']
    cols = ['number_factors', 'Combi', 'AUC_score', 'Intercept'] + list(list_factors)
    mymodel = LogisticRegression()
    mymodel.fit(x_sample, y_sample)
    vals=[len(list_factors)
          ,list_factors
           ,roc_auc_score(y_sample, mymodel.predict_proba(x_sample)[:,1])
           ,mymodel.intercept_[0]] + list(mymodel.coef_[0]*-1)
    mydict = dict(zip(cols, vals))
    return mydict

In [94]:
lstModel=[]

import time
start = time.process_time()

for i, row in dfCombi.iterrows():
    lstModel.append(run_logistic_regression(dfNormDev, row['CombiFactors']))

print(f'It took {time.process_time() - start} seconds')
# 187.578125 ms? for this sequence run

MFA_result=pd.DataFrame(lstModel)
MFA_result.to_csv('MFA_result.csv', index=False)

It took 604.71875 seconds


In [95]:
MFA_result.iloc[MFA_result['AUC_score'].idxmax()] 

number_factors                                                                   11
Combi                             (LTV_NS, CODE_GENDER_NS, YRS_EMPLOYED_NS, NAME...
AUC_score                                                                   0.72849
Intercept                                                                  -2.75668
LTV_NS                                                                   0.00372469
CODE_GENDER_NS                                                           0.00217398
AGE_NS                                                                          NaN
YRS_EMPLOYED_NS                                                          0.00285478
OCCUPATION_TYPE_NS                                                              NaN
NAME_EDUCATION_TYPE_NS                                                    0.0031474
EXT_SOURCE_1_NS                                                          0.00339984
EXT_SOURCE_2_NS                                                          0.0

## Gán điểm cuối cùng

Sử dụng file excel để lựa chọn mô hình cuối. Sau đó copy trọng số vào đây 


In [103]:
mapNormScore

scaler = 20/np.log(2)
offset = 600 - scaler*np.log(4)
nFactors = 11

intercept =-2.756676623


dictCoef = {    
    'LTV': 0.00429033,
    'CODE_GENDER': 0.00273021,
    'YRS_EMPLOYED': 0.00351027,
    'NAME_EDUCATION_TYPE': 0.0039003,
    'YRS_ID_PUBLISH':0.008190631,
    'NAME_INCOME_TYPE':0.00234018,
    'YRS_LAST_PHONE_CHANGE': 0.0039003,
    'EXT_SOURCE_1': 0.0039003,
    'EXT_SOURCE_2': 0.00195015,
    'EXT_SOURCE_3': 0.00234018,
    'TOTALAREA_MODE':0.00234018
    
}




mapNormScore['FactorScore'] = round(mapNormScore['NormScore']*(mapNormScore['FactorName'].map(lambda x: dictCoef.get(x)))*scaler 
                                    + (1/nFactors)*(intercept*scaler+offset),0)


dfNormDev['TotalScore']=0
for factor, coef in dictCoef.items():
    dfNormDev[factor+'_score'] = round(dfNormDev[factor + '_NS']*coef*scaler + (1/nFactors)*(intercept*scaler+offset),0)
    dfNormDev['TotalScore'] += dfNormDev[factor+'_score']
    
dfNormDev.to_csv(f"dfNormDev.csv", index=False)

In [97]:
dfNormDev

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE_bucket,NAME_CONTRACT_TYPE_NS,CODE_GENDER,CODE_GENDER_bucket,CODE_GENDER_NS,LTV,LTV_bucket,...,CODE_GENDER_score,YRS_EMPLOYED_score,NAME_EDUCATION_TYPE_score,YRS_ID_PUBLISH_score,NAME_INCOME_TYPE_score,YRS_LAST_PHONE_CHANGE_score,EXT_SOURCE_1_score,EXT_SOURCE_2_score,EXT_SOURCE_3_score,TOTALAREA_MODE_score
0,156852,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.32,03. >1.28 or missing,...,46.0,38.0,40.0,45.0,40.0,50.0,36.0,44.0,39.0,41.0
1,211489,0,Cash loans,01. Cash,-16.44,M,02. Male,-70.01,1.00,01. <=1.15,...,38.0,44.0,40.0,27.0,47.0,50.0,47.0,44.0,42.0,41.0
2,448618,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.16,"02. (1.15, 1.28]",...,46.0,51.0,40.0,61.0,47.0,39.0,42.0,48.0,45.0,47.0
3,401051,0,Cash loans,01. Cash,-16.44,F,01. Female,35.71,1.22,"02. (1.15, 1.28]",...,46.0,44.0,40.0,45.0,40.0,50.0,55.0,44.0,39.0,47.0
4,215007,0,Cash loans,01. Cash,-16.44,M,02. Male,-70.01,1.15,01. <=1.15,...,38.0,38.0,40.0,45.0,40.0,39.0,55.0,40.0,39.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30746,345491,0,Revolving loans,02. Revolving,152.10,M,02. Male,-70.01,1.00,01. <=1.15,...,38.0,38.0,41.0,27.0,47.0,39.0,36.0,44.0,45.0,47.0
30747,359345,0,Revolving loans,02. Revolving,152.10,M,02. Male,-70.01,1.00,01. <=1.15,...,38.0,38.0,54.0,45.0,47.0,39.0,42.0,48.0,49.0,41.0
30748,137463,0,Revolving loans,02. Revolving,152.10,F,01. Female,35.71,1.00,01. <=1.15,...,46.0,44.0,40.0,45.0,47.0,50.0,42.0,44.0,49.0,50.0
30749,309635,0,Revolving loans,02. Revolving,152.10,F,01. Female,35.71,1.00,01. <=1.15,...,46.0,51.0,40.0,45.0,40.0,39.0,55.0,44.0,49.0,41.0


In [98]:
mapNormScore

,FactorName,FinalBucketName,NormScore,FactorScore
0,AGE,01. <=30,-62.20,NaN
1,AGE,"02. (30,50]",-25.16,NaN
2,AGE,"03. (50,60)",51.63,NaN
3,AGE,04. 60<,94.41,NaN
4,APARTMENTS_AVG,01. missing,-39.73,NaN
5,APARTMENTS_AVG,"02. (0,0.13]",13.98,NaN
6,APARTMENTS_AVG,03. >0.13,106.21,NaN
7,CODE_GENDER,01. Female,35.71,46.0
8,CODE_GENDER,02. Male,-70.01,38.0
9,EXT_SOURCE_1,"01. (0, 0.4]",-65.84,36.0


In [99]:
finale = mapNormScore.dropna()
finale

,FactorName,FinalBucketName,NormScore,FactorScore
7,CODE_GENDER,01. Female,35.71,46.0
8,CODE_GENDER,02. Male,-70.01,38.0
9,EXT_SOURCE_1,"01. (0, 0.4]",-65.84,36.0
10,EXT_SOURCE_1,02. missing,-17.27,42.0
11,EXT_SOURCE_1,"03. (0.4, 0.615)",30.79,47.0
12,EXT_SOURCE_1,04. 0.615<,102.16,55.0
13,EXT_SOURCE_2,"01. (0, 0.1]",-144.16,36.0
14,EXT_SOURCE_2,"02. (0.1,0.4]",-57.63,40.0
15,EXT_SOURCE_2,"03. (0.4, 0.7]",10.21,44.0
16,EXT_SOURCE_2,04. mising,58.10,47.0


In [109]:
dfBscore = RiskRankingTable(dfNormDev, 'TotalScore', [0, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 999])
AR, dfAR = AR_Calculator(dfBscore, 'Total', 'Bad')
print(f'Model AR: {AR}')
dfBscore

Model AR: 0.3535019343985321


,FactorName,Bucket,Total,Bad,Dist,BadRate,RBR
0,TotalScore,"00. (400, 410]",10,3,0.000325,0.300000,3.802679
1,TotalScore,"01. (410, 420]",83,23,0.002699,0.277108,3.512515
2,TotalScore,"02. (420, 430]",396,90,0.012878,0.227273,2.880818
3,TotalScore,"03. (430, 440]",1043,192,0.033918,0.184084,2.333379
4,TotalScore,"04. (440, 450]",2067,324,0.067217,0.156749,1.986886
5,TotalScore,"05. (450, 460]",3277,446,0.106566,0.136100,1.725150
6,TotalScore,"06. (460, 470]",4348,400,0.141394,0.091996,1.166108
7,TotalScore,"07. (470, 480]",4841,351,0.157426,0.072506,0.919053
8,TotalScore,"08. (480, 490]",4912,262,0.159735,0.053339,0.676101
9,TotalScore,"09. (490, 500]",4008,170,0.130337,0.042415,0.537638
